In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, transforms
import cv2 as cv
import os
from pathlib import Path
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
dataset_path = Path('./Dataset/')
train_data_path = dataset_path / "train dataset"
test_data_path = dataset_path / "test dataset"

In [3]:
!unzip Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/train dataset/id_5499_label_16.png  
  inflating: Dataset/train dataset/id_55_label_7.png  
  inflating: Dataset/train dataset/id_550_label_13.png  
  inflating: Dataset/train dataset/id_5500_label_16.png  
  inflating: Dataset/train dataset/id_5501_label_16.png  
  inflating: Dataset/train dataset/id_5502_label_16.png  
  inflating: Dataset/train dataset/id_5503_label_16.png  
  inflating: Dataset/train dataset/id_5504_label_16.png  
  inflating: Dataset/train dataset/id_5505_label_17.png  
  inflating: Dataset/train dataset/id_5506_label_17.png  
  inflating: Dataset/train dataset/id_5507_label_17.png  
  inflating: Dataset/train dataset/id_5508_label_17.png  
  inflating: Dataset/train dataset/id_5509_label_17.png  
  inflating: Dataset/train dataset/id_551_label_13.png  
  inflating: Dataset/train dataset/id_5510_label_17.png  
  inflating: Dataset/train dataset/id_5511_label_17.png  
  inflating: Dataset/train

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:

def upload_images(train_data_path, test_data_path):
    train_data = []
    test_data = []
    for filename in list(os.listdir(train_data_path)):
        train_data.append((cv.imread(str(train_data_path/filename), cv.IMREAD_GRAYSCALE), int(filename.split(".")[0].split("_")[-1]) - 1))
    for filename in list(os.listdir(test_data_path)):
        test_data.append((cv.imread(str(test_data_path/filename), cv.IMREAD_GRAYSCALE), int(filename.split(".")[0].split("_")[-1]) - 1))

    return train_data, test_data

In [6]:
train_data, test_data = upload_images(train_data_path, test_data_path)

In [7]:
train_dataloader = DataLoader(dataset=train_data, batch_size=32)
test_dataloader = DataLoader(dataset=test_data, batch_size=32)

In [8]:
class CNNModelV0(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=10),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=10, out_channels=40, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=40),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=40, out_channels=80, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=80),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(in_features=4 * 4 * 80, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=28)
        )
    def forward(self, x):
        x = self.cnn_layer(x)
        x = torch.flatten(x, 1)
        x = self.fc_layer(x)
        return x
model_0 = CNNModelV0().to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.001, params=model_0.parameters())

In [10]:
epochs = 10

for epoch in range(epochs):
    model_0.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data

        inputs = inputs.to(device).float().unsqueeze(1)
        labels = labels.to(device)
        outputs = model_0(inputs)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()


In [12]:
with torch.inference_mode():
  model_0.eval()
  total_correct = 0
  total_samples = 0
  for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    inputs = inputs.to(device).float().unsqueeze(1)
    labels = labels.to(device)
    outputs = F.softmax(model_0(inputs), 1)

    _, predicted_labels = torch.max(outputs, 1)
    total_correct += (predicted_labels == labels).sum().item()
    total_samples += labels.size(0)
  test_accuracy = total_correct / total_samples
  print(f'Average Test Accuracy: {test_accuracy * 100:.2f}%')

Average Test Accuracy: 90.21%
